In [1]:
import pandas as pd
from itertools import cycle
import gpflow
import numpy as np
from gpflow.utilities import print_summary, positive
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import namedtuple
import statsmodels.api as sm
import dill as pickle

sns.set(style="white")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

gpflow.config.set_default_float(np.float64)
gpflow.config.set_default_summary_fmt("notebook")

/Users/leo/.pyenv/versions/3.8.7/envs/covid/lib/python3.8/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
<ipython-input-1-06da3225ef62>:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
final_data = pd.read_pickle("tmp/final_data.pkl")
u0 = pd.read_pickle("tmp/u0.pkl")
linear_filters_states = pickle.load(open('tmp/linear_filters_states.pkl', 'rb'))

## Rhos

In [3]:
ORDER = len(list(linear_filters_states.values())[0][0])
N = len(linear_filters_states)

In [4]:
times = None
for s in final_data.index.unique():
    s_times = set(final_data.loc[s, 'Updated'].unique())
    if times is None:
        times = s_times
    else:
        times = times.intersection(s_times)

In [5]:
u0_ = u0[u0['Updated'].isin(times)]['Residual'].values
T = u0_.shape[0]
t = np.arange(1, T + 1).reshape(-1, 1)

T2 = T - ORDER + 1

rhos = np.zeros((T2*N, T2*N))
for i, (hX, epsX) in enumerate(linear_filters_states.values()):
    for j, (hY, epsY) in enumerate(linear_filters_states.values()):
        convol = np.convolve(hX, hY[::-1], mode="same")
        for t in range(T2):
            data = convol[max(len(convol)//2 - t, 0): min(len(convol), T2 - t + len(convol)//2)]
            jstart = j*T2+max(t - len(convol)//2, 0)
            jend = jstart + len(data)
            rhos[i*T2+t, jstart:jend] = data

In [6]:
# validation
for x in range(N):
    for y in range(N):
        for t in range(T2):
            for s in range(T2):
                assert rhos[x*T2+t, y*T2+s] == rhos[y*T2+s, x*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{XY}}_{{t,s}} = \rho^{{YX}}_{{s,t}}"
                assert rhos[y*T2+t, x*T2+s] == rhos[x*T2+s, y*T2+t], \
                f"x={x}, y={y}, t={t}, s={s} => \rho^{{YX}}_{{t,s}} = \rho^{{XY}}_{{s,t}}"

In [7]:
rhos.T == rhos

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [8]:
rhos

array([[ 0.29698109,  0.03198337, -0.05212048, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03198337,  0.29698109,  0.03198337, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05212048,  0.03198337,  0.29698109, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.13562352,
         0.04408787, -0.03850037],
       [ 0.        ,  0.        ,  0.        , ...,  0.04408787,
         0.13562352,  0.04408787],
       [ 0.        ,  0.        ,  0.        , ..., -0.03850037,
         0.04408787,  0.13562352]])

In [9]:
rhos.shape

(2516, 2516)